In [ ]:
import os
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from nltk.sentiment import SentimentIntensityAnalyzer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Load dataset
df = pd.read_csv("news_scraping_results.csv")

# Cek jumlah data
print(f"Jumlah data: {len(df)}")
if len(df) < 10000:
    print("⚠️ Dataset masih kurang dari 10.000! Tambahkan lebih banyak data.")

# Data Cleaning
df = df.dropna().reset_index(drop=True)

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z ]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

df['clean_content'] = df['content'].apply(clean_text)

# Sentiment Labeling
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    score = sia.polarity_scores(text)
    if score['compound'] > 0.05:
        return 'positive'
    elif score['compound'] < -0.05:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['clean_content'].apply(get_sentiment)

# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(df['clean_content'], df['sentiment'], test_size=0.2, random_state=42)

# TF-IDF Vectorization (untuk SVM dan RF)
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Model 1: Support Vector Machine (SVM)
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm_model = GridSearchCV(SVC(), svm_params, cv=5)
svm_model.fit(X_train_tfidf, y_train)
svm_preds = svm_model.predict(X_test_tfidf)
print("SVM Best Params:", svm_model.best_params_)
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))

# Model 2: Random Forest
rf_model = RandomForestClassifier(n_estimators=300, max_depth=30, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
rf_preds = rf_model.predict(X_test_tfidf)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))

# Model 3: Improved LSTM Deep Learning
max_words = 15000  # Menambah jumlah kata unik
max_len = 250  # Memperpanjang panjang maksimum sequence

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

# Model LSTM
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    Conv1D(128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(256, return_sequences=True)),  # Naikkan ke 256 unit
    BatchNormalization(),
    LSTM(128, return_sequences=False),  # Naikkan ke 128 unit
    Dense(128, activation='relu'),
    Dropout(0.5),  # Meningkatkan dropout untuk mengurangi overfitting
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Learning Rate Scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

history = model.fit(X_train_seq, y_train.factorize()[0], epochs=40, batch_size=32, validation_data=(X_test_seq, y_test.factorize()[0]), callbacks=[lr_scheduler])

# Evaluasi Model
lstm_accuracy = model.evaluate(X_test_seq, y_test.factorize()[0], verbose=0)[1]
print("LSTM Accuracy:", lstm_accuracy)

# Plot Training & Validation Accuracy
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot Training & Validation Loss
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
